# langsmith简介

LangSmith 是 LangChain 提供的 AI 应用开发监测平台，我们可以用它来观察调用链的运行情况。
- 官方网站：https://smith.langchain.com/
- 官方文档：https://docs.smith.langchain.com/

# 快速上手

## 依赖安装与配置

1. LangSmith安装
```shell
pip install -U langsmith openai 
```
2. 创建api-key
在[设置](https://smith.langchain.com/settings)页面创建api-key

3. 配置环境变量(也可以在代码中设置)
```shell
export LANGCHAIN_TRACING_V2=true
export LANGCHAIN_API_KEY=<your-api-key>

# The below examples use the OpenAI API, though it's not necessary in general
export OPENAI_API_KEY=<your-openai-api-key>
```

![](9.png)
![](10.png)

## 示例

In [1]:
import os
from dotenv import dotenv_values

env_config = dotenv_values("../.env")
# env_config

In [2]:
os.environ["LANGCHAIN_TRACING_V2"]= "true"
os.environ["LANGCHAIN_ENDPOINT"] ="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]=env_config.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]="pr-timely-sector-47"
# os.environ["OPENAI_API_KEY"] = "..."
os.environ["ZHIPUAI_API_KEY"] = env_config.get("ZHIPUAI_API_KEY")

#### 加载依赖

In [3]:
from langsmith.evaluation import EvaluationResult, run_evaluator
from langchain.smith import RunEvalConfig, run_on_dataset
# from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from IPython.display import Markdown
from langchain_community.chat_models import ChatZhipuAI

#### 记录调用链

In [4]:

llm = ChatZhipuAI()


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")  
])

translate_prompt = ChatPromptTemplate.from_messages([
    ("system", "Translate to simplified Chinese."),
    ("user", "{input}")
])

output_parser = StrOutputParser()

chain = prompt | llm | output_parser \
        | {"input": RunnablePassthrough()} \
        | translate_prompt | llm | output_parser

output = chain.invoke({"input": "what's generative ai?"})

In [5]:
output

'生成式人工智能是指涉及生成新数据、内容或媒体的人工智能（AI）技术的一个子集。这些算法能够根据从给定数据集中学习到的模式和信息创建新的输出，如文本、图像、音乐、代码或任何其他形式的数据。\n\n生成式AI模型有几种类型，包括：\n\n1. **生成对抗网络（GANs）**：由两个神经网络组成，一个是生成器，一个是判别器，通过竞争性过程同时进行训练。生成器试图产生看起来真实的数据，而判别器则尝试区分真实数据和生成数据。\n\n2. **变分自编码器（VAEs）**：这是一种深度学习模型，它学习将输入数据编码为更小、更紧凑的表示形式，然后从该表示生成新的数据。\n\n3. **循环神经网络（RNNs）**：特别是长短期记忆（LSTM）网络或门控循环单元（GRUs），适合生成诸如文本或音乐之类的序列数据。\n\n4. **基于Transformer的模型**：这些模型特别适用于语言生成，包括像OpenAI的GPT（生成预训练Transformer）和BERT（双向编码器表示的Transformer）这样的模型。\n\n生成式AI已经在各种领域找到了应用，例如：\n\n- **内容创作**：生成文章、故事或音乐。\n- **艺术与设计**：创作新的图像、数字艺术和动画。\n- **数据增强**：通过创建新的、逼真的样本来增强数据集，以提高模型训练的效果。\n- **药物发现**：合成具有所需特性的新分子。\n- **软件开发**：自动生成代码或帮助调试。\n\n作为一名技术文档编写者，生成式AI工具对于自动化重复性或样板内容的生成特别有用，有助于组织和管理复杂的文档结构，或根据现有内容提供文本建议。'

![](2.png)

![](1.png)

#### 在某个数据集上评估AI应用

LangSmith 的另一个常见用途是在数据集上运行评估。当我们开发一个 AI 应用的调用链后，我们要运行评估，以判断它是否符合预期。

LangSmith 集成了 LangChain 的评估功能，为我们提供了一个系统化的 AI 调用链评估工具：

1. 开发一个 AI 应用的调用链（即业务逻辑）
2. 在 LangSmith 创建数据集
3. 在数据集上运行评估
4. 在 LangSmith 界面查看评估结果
以下示例选自 LangSmith 测试与评估 中的调用链部分。略作修改，主要是增加部分解释。

##### 开发一个AI调用链

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
# from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system","请分析下面的用户评论，并确定它是积极还是消极。如果是积极，用“好评”回答，如果是消极，用“差评”回答。"),
    ("human","{comment}"),  # 输入用户评论
])
chain = prompt | llm | StrOutputParser()

In [7]:
chain.invoke("真不是人住的地方")

'差评'

这个提示语要求 AI 模型分析用户评论，输入参数为comment。

##### 创建评估数据集

![](3.png)

我们可以用如下代码创建数据集，但更方便的方式是直接在 LangSmith 界面中直接创建。

In [8]:
#包含input输入的数据和output输出的对应标签
examples = [
    {
    "comment": "这家酒店环境很好,交通便利,服务员非常友好,下次还会来!",
    "reviews": "好评"
    },
    {
    "comment": "房间虽小但很干净整洁,性价比很高,下次还会选择这里。",
    "reviews": "好评"
    },
    {
    "comment": "酒店的游泳池和健身房设施一流,住宿体验非常棒,下次有机会一定再来。",
    "reviews": "好评"
    },
    {
    "comment": "早餐种类很丰富,味道也不错,服务态度很好,对得起这个价格。",
    "reviews": "好评"
    },
    {
    "comment": "酒店位置非常优越,临近著名景点,出行非常方便,下次有机会一定会再入住。",
    "reviews": "好评"
    },
    {
    "comment": "房间设施有些陈旧,卫生情况一般,不太符合我的预期,下次不会再选择这里。",
    "reviews": "差评"
    },
    {
    "comment": "服务员的态度不太友好,办理入住和退房的效率也比较低下,不太满意。",
    "reviews": "差评"
    },
    {
    "comment": "酒店停车场收费很高,且位置不太好找,感觉性价比不太高,不会再来了。",
    "reviews": "差评"
    },
    {
    "comment": "房间隔音效果很差,能听到走廊和邻居的声音,影响睡眠质量,希望能够改进。",
    "reviews": "差评"
    },
    {
    "comment": "酒店餐厅的菜品种类和口味都一般,性价比不高,下次不会再选择这里。",
    "reviews": "差评"
    }
]

In [ ]:
from langsmith import Client

inputs=[]
outputs=[]
for example in examples:
    inputs.append({'comment':example['comment']})
    outputs.append({'reviews':example['reviews']})

# #创建数据集
client = Client()
dataset_name = "酒店评论数据集3"  # 数据集名字
dateset = client.create_dataset(dataset_name,description="Hotel reviews dataset for comment and query")  # 创建数据集
client.create_examples(dataset_id=dateset.id,inputs=inputs,outputs=outputs)  

![](4.png)

如上代码将创建一个评估数据集，其中一条数据如下图。在其中我们可看到输入与输出。

![](5.png)

##### 创建评估

In [13]:
# help(RunEvalConfig)

In [11]:
@run_evaluator
def correct_label(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""    #调用链返回的预测值
    required = example.outputs.get("reviews") or "" #数据集中的标签
    score = prediction == required  # 相等为1， 不同为0
    return {"score": int(score), "key": "correct_label"}

eval_config = RunEvalConfig(
    custom_evaluators=[correct_label],  # 自定义评估器
)

# dataset_name = "Hotel Reviews Dataset"
client = Client()

client.run_on_dataset(
    dataset_name=dataset_name,  # 数据集
    llm_or_chain_factory=chain,  # 设置要测试的 chain
    evaluation=eval_config,  # 评估
    verbose=True,
    project_name="Hotel Reviews Project3",  # 评估的名字
    project_metadata={"version": "1.0.0"},
)

View the evaluation results for project 'Hotel Reviews Project3' at:
https://smith.langchain.com/o/14adf85e-d828-472b-a1b1-6e1ccbad9da5/datasets/076e4577-a8ef-4e93-90d6-969533cf088b/compare?selectedSessions=96ea260a-409e-4494-a466-0fa9d8b6d2b9

View all tests for Dataset 酒店评论数据集3 at:
https://smith.langchain.com/o/14adf85e-d828-472b-a1b1-6e1ccbad9da5/datasets/076e4577-a8ef-4e93-90d6-969533cf088b
[------------------------------------------------->] 10/10

{'project_name': 'Hotel Reviews Project3',
 'results': {'3d252a33-a3f6-4157-b0bb-cfba75c57912': {'input': {'comment': '酒店餐厅的菜品种类和口味都一般,性价比不高,下次不会再选择这里。'},
   'feedback': [EvaluationResult(key='correct_label', score=1, value=None, comment=None, correction=None, evaluator_info={}, feedback_config=None, source_run_id=UUID('8c0a0fcc-c63d-40cc-9c1d-b7fbeb9db9ca'), target_run_id=None, extra=None)],
   'execution_time': 0.363384,
   'run_id': '1d76f18d-eedb-4298-8fb5-d8e11d5183dd',
   'output': '差评',
   'reference': {'reviews': '差评'}},
  '3799785c-1d30-40a2-8099-495c0417a96e': {'input': {'comment': '房间隔音效果很差,能听到走廊和邻居的声音,影响睡眠质量,希望能够改进。'},
   'feedback': [EvaluationResult(key='correct_label', score=1, value=None, comment=None, correction=None, evaluator_info={}, feedback_config=None, source_run_id=UUID('dca3c034-fd3e-4964-8cbc-926864fbc451'), target_run_id=None, extra=None)],
   'execution_time': 0.562086,
   'run_id': '43e9c4a3-636a-4137-b4ff-cbcb60aa9ce6',
   'output': '差评',
   'reference': {

##### 查看评估结果

我们可在 LangSmith 界面中查看评估结果，如下图所示。

![](6.png)

可以发现输出结果跟预期不一致（期望输出“差评”，实际输出“差评。”），因此需要prompt优化
> 请分析下面的用户评论，并确定它是积极还是消极。如果是积极，只需要回答“好评”两字，如果是消极，只需要回答“差评”两字。

In [12]:
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
prompt = ChatPromptTemplate.from_messages([
    ("system","请分析下面的用户评论，并确定它是积极还是消极。如果是积极，只需要回答“好评”两字，如果是消极，只需要回答“差评”两字。"),
    ("human","{comment}"),
])
chain = prompt | llm | StrOutputParser()

@run_evaluator
def correct_label(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""    #调用链返回的预测值
    required = example.outputs.get("reviews") or "" #数据集中的标签
    score = prediction == required
    return {"score": int(score), "key": "correct_label"}

eval_config = RunEvalConfig(
    custom_evaluators=[correct_label],  # 自定义评估器
)

dataset_name = "Hotel Reviews Dataset"
client = Client()

client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=chain,
    evaluation=eval_config,
    verbose=True,
    project_name="Hotel Reviews Project4",
    project_metadata={"version": "1.0.0"},
)

View the evaluation results for project 'Hotel Reviews Project4' at:
https://smith.langchain.com/o/14adf85e-d828-472b-a1b1-6e1ccbad9da5/datasets/b35712d2-2e74-44f6-b5b8-c1fdaed07356/compare?selectedSessions=f844c51b-ace3-4ea7-aadb-356a1714b448

View all tests for Dataset Hotel Reviews Dataset at:
https://smith.langchain.com/o/14adf85e-d828-472b-a1b1-6e1ccbad9da5/datasets/b35712d2-2e74-44f6-b5b8-c1fdaed07356
[------------------------------------------------->] 10/10

{'project_name': 'Hotel Reviews Project4',
 'results': {'2e260358-5bec-4750-a719-bbd967c1bae6': {'input': {'comment': '酒店餐厅的菜品种类和口味都一般,性价比不高,下次不会再选择这里。'},
   'feedback': [EvaluationResult(key='correct_label', score=1, value=None, comment=None, correction=None, evaluator_info={}, feedback_config=None, source_run_id=UUID('111fbfb6-a80e-4b2c-b1e9-0f2354204752'), target_run_id=None, extra=None)],
   'execution_time': 0.405122,
   'run_id': '8493d33b-0e61-49b4-9331-f2f832778a40',
   'output': '差评',
   'reference': {'reviews': '差评'}},
  'f0001b29-31e7-4b02-a786-5c2ec9dc7d8c': {'input': {'comment': '房间隔音效果很差,能听到走廊和邻居的声音,影响睡眠质量,希望能够改进。'},
   'feedback': [EvaluationResult(key='correct_label', score=1, value=None, comment=None, correction=None, evaluator_info={}, feedback_config=None, source_run_id=UUID('13258e4b-5c39-4986-98a5-ac8d855f75da'), target_run_id=None, extra=None)],
   'execution_time': 0.513039,
   'run_id': '6b8d3a40-14c5-4b52-b71f-accf4b0c4c3b',
   'output': '差评',
   'reference': {

![](8.png)

![](9.png)

### 小结

通过如上讨论，我们可以看到，LangSmith 为我们观测 AI 应用提供了多种便利的工具，这同样能加速 AI 应用的开发。